# Modelling dam volumes using DE Africa waterbodies
# Section 01  : *Data Preprocessing*

**Products used:** 
[DE Africa Waterbodies](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html), 
[Department of Water Affairs and Sanitation, South Africa Dam Level and Volume Data](https://www.dws.gov.za/Hydrology/Verified/hymain.aspx)

## Background

### Digital Twin (DT)
The CGIAR Digital Twin initiative creates dynamic virtual models that combine real-time data, AI, and simulations to improve decision-making. Its prototype for the Limpopo River Basin focuses on enhancing water resource management and conservation.


## Description
This notebook presents a workflow for predicting dam levels and volumes using water surface area data from DE Africa's Waterbodies product, integrating data preprocessing, feature extraction, and Gradient Boosting modeling. 

As part of the CGIAR Initiative on Digital Innovation, this work contributes to a prototype Digital Twin for the Limpopo River Basin, designed to support real-time decision-making in water management. The Digital Twin leverages AI-driven tools to visualize and simulate the impact of decisions on the basin's ecosystem. To enhance prediction reliability, the model includes a correction mechanism to address unrealistic large drops in dam volume estimates.


## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
import datacube
import joblib

from scipy import interpolate
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go

from deafrica_tools.waterbodies import get_waterbody, get_time_series, display_time_series,get_geohashes
from IPython.display import Image

### Analysis parameters
This section defines the analysis parameters, including:
* `waterbody_geohash`: Unique identifier (uid) / [geohash](https://en.wikipedia.org/wiki/Geohash) a waterbody.  The geohash of a water body is derived from its position, and this process can be reversed to obtain the location from the geohash. A waterbody's geohash is contained under the `uid` attribute and can be obtained through DE Africa Maps by clicking on a [waterbody](https://maps.digitalearth.africa/waterbody).

For this model development we will train and test for Loskop dam in South Africa with a DE Africa Water Body Geohash of:


In [ ]:
waterbody_geohash = "kekz70yc3g"

## Get data for a specific waterbody

The returned GeoDataFrame includes the metadata for the selected waterbody including the id, uid, wb_id, area, perimeter and length. See the [Waterbodies Historical Extent documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html#Waterbodies-Historical-Extent) for descriptions of each attribute.

In [ ]:
waterbody = get_waterbody(waterbody_geohash)
waterbody

Once the water body polygon is in memory, you can plot them directly, or explore them in an interactive window.

In [ ]:
waterbody.plot()

In [ ]:
waterbody.explore()

## Get the wet surface area time series for the selected waterbody

For any given waterbody, we can also use the `get_time_series()` function to get various measures of the water body surface over time. See the [Waterbodies Historical Extent documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html#Waterbodies-Surface-Area-Change) for descriptions of the different surface measures.

The function also calculates a rolling median of the water body surface wet percentage. This is used to visualise the overall trend in the surface wet percentage. The rolling median uses the last three observations to determine the median at a given date.

> By default the entire timeseries for the waterbody i.e. `start_date=1984-01-01` and `end_date=today` is loaded for the waterbody by the `get_time_series()` function  unless a filter is applied by passing the `start_date` and `end_date` parameters to the function.

In [ ]:
waterbody_timeseries = get_time_series(waterbody=waterbody)
waterbody_timeseries

### Display the wet surface area time series for the selected waterbody

After loading the water body time series, we can use the `display_time_series()` function to create an interactive visualisation of the time series.

The visualisation shows the invalid percentage and the wet percentage. The invalid percentage indicates the proportion of the water body that couldn't be clearly observed. To provide the most representative measurements of water body surface area, the time series only contains values where the invalid percentage is lower than 10%.

There are some caveats to be aware of:

- To appear in the time series, an observation must record clear observations of at least 90% of the water body's surface area. If 10% or more of the surface area is covered by cloud or cloud shadow, the observation will be excluded. This can cause large gaps in the time series.
- If the invalid percentage is high, it's likely that the wet percentage is an underestimate of the true wet surface area.
- Annual and seasonal trends should only be inferred during times with sufficient observations. You should take care when infering the surface water change across time when there are few observations.
- The time series is based on the Water Observations from Space product, which has known limitations. See the [DE Africa Waterbodies service documentation](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html) for more information.

In [ ]:
# Create a copy of the DataFrame with the index reset
waterbody_timeseries_plot = waterbody_timeseries.reset_index()

# Create the figure and axis
fig, ax = plt.subplots(figsize=(14, 5))

# Plot Wet Percentage
ax.scatter(waterbody_timeseries_plot['date'], waterbody_timeseries_plot['percent_wet'], 
           color='blue', label='Wet Percentage', s=20, marker='o')

# Plot Invalid Percentage
ax.scatter(waterbody_timeseries_plot['date'], waterbody_timeseries_plot['percent_invalid'], 
           color='red', label='Invalid Percentage', s=20, marker='o')

# Plot Rolling Median Wet Percentage
ax.plot(waterbody_timeseries_plot['date'], waterbody_timeseries_plot['percent_wet_rolling_median'], 
        color='blue', linewidth=1, label='Wet Percentage - Rolling Median')

# Set labels and title
ax.set_title('Wet Surface Area Time Series')
ax.set_xlabel('Date')
ax.set_ylabel('Percentage')
plt.xticks(rotation=45)
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(title='Legend', loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

# Save the figure as a PNG image
# image_path = "wet_surface_area_time_series_matplotlib.png"
# plt.savefig(image_path, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

## Load Model Training Datasets

#### Function to convert waterbody_timeseries into hectares based datatable 

In [ ]:
def convert_to_hectare_table(waterbody_timeseries):
    # Create water_area_ha by dividing area_wet_m2 by 10,000 (conversion from square meters to hectares)
    waterbody_timeseries['water_area_ha'] = waterbody_timeseries['area_wet_m2'] / 10000

    # Prepare final table
    digital_twin_table = pd.DataFrame({
        'original_id': ['kekz70yc3g'] * len(waterbody_timeseries),
        'id': range(183875, 183875 + len(waterbody_timeseries)),
        'waterbody_id': [518] * len(waterbody_timeseries),  # Assuming a constant waterbody_id as per Limpopo Digital Twin
        'date': waterbody_timeseries.index,  # Assuming 'date' is the index of the waterbody_timeseries
        'water_area_ha': waterbody_timeseries['water_area_ha'],
        'percent_invalid': waterbody_timeseries['percent_invalid']
    })
    return digital_twin_table

# Call the conversion function on the waterbody_timeseries to replace the old water levels data loading
water_areas_ha = convert_to_hectare_table(waterbody_timeseries)
water_areas_ha


### Sample Data Overview

This dataset contains raw water levels data collected from DEA (Department of Environmental Affairs) in South Africa. The cell below reads this ancillary data necessary to conduct the volume prediction.

In [ ]:
dam_attributes = pd.read_csv('data/dam_attributes/dam_attributes.csv')
rating_curve = pd.read_csv('data/DWS/rating_curve.csv')
daily_volumes = pd.read_csv('data/DWS/daily_volumes.csv')

dam_attributes

#### Extract dam attributes
The full value indicator i.e. the value representing the dam at 100% capacity for both the volume and the area were found to be a strong feature for training the model and acting as a baseline starting point.

In [ ]:
full_volume = dam_attributes[dam_attributes['parameter'] == 'full_volume']['value'].values[0]
full_surface_area = dam_attributes[dam_attributes['parameter'] == 'full_surface_area']['value'].values[0]
full_dam_level = dam_attributes[dam_attributes['parameter'] == 'full_dam_level']['value'].values[0]
maximum_dam_level = dam_attributes[dam_attributes['parameter'] == 'maximum_dam_level']['value'].values[0]
capacity_of_spillway = dam_attributes[dam_attributes['parameter'] == 'capacity_of_spillway']['value'].values[0]
vertical_drop = dam_attributes[dam_attributes['parameter'] == 'vertical_drop']['value'].values[0]
mean_depth = dam_attributes[dam_attributes['parameter'] == 'mean_depth']['value'].values[0]
full_capacity_elevation = dam_attributes[dam_attributes['parameter'] == 'full_capacity_elevation']['value'].values[0]
shoreline_length = dam_attributes[dam_attributes['parameter'] == 'shoreline_length']['value'].values[0]
maximum_volume = dam_attributes[dam_attributes['parameter'] == 'maximum_volume']['value'].values[0]
maximum_surface_area = dam_attributes[dam_attributes['parameter'] == 'maximum_surface_area']['value'].values[0]

In [ ]:
rating_curve

In [ ]:
daily_volumes

### Data Preprocessing

In the data preprocessing stage, it was crucial to handle potential outliers in the water area dataset, which could negatively impact model accuracy. We applied the **Interquartile Range (IQR) method** to systematically identify and remove these outliers.

The IQR method flags data points that lie beyond 1.5 times the interquartile range (IQR), calculated as the distance between the first quartile (Q1) and the third quartile (Q3). Any values falling below Q1 or above Q3 by more than 1.5 times the IQR were considered outliers. Removing these extreme values helped ensure that the model was trained on reliable, realistic water area measurements, improving its robustness and generalizability.

To account for seasonal variability, the IQR method was applied on a per-month basis, ensuring that seasonal changes did not distort the detection process.

**Key steps:**
- Remove outliers per month using the IQR method.
- Handle missing values by dropping rows containing `NaN` in critical feature columns, such as `water_area_ha`.

---

#### Filter water levels based on percentage invalid
- Filter out rows where the percent_invalid is greater than 10.
- Calculate and print the number of samples removed during the filtering process.
- Print the number of samples remaining in the dataset after the filtering.

In [ ]:
initial_count = len(water_areas_ha)
water_areas_filtered = water_areas_ha[water_areas_ha['percent_invalid'] <= 0.5]
final_count = len(water_areas_filtered)
removed_count = initial_count - final_count
print(f"Number of samples removed: {removed_count}")
print(f"Number of samples remaining: {final_count}")

#### Convert date columns to datetime

In [ ]:
water_areas_filtered = water_areas_filtered.copy()
daily_volumes = daily_volumes.copy()
water_areas_filtered['date'] = pd.to_datetime(water_areas_filtered['date'])
daily_volumes['Date'] = pd.to_datetime(daily_volumes['Date'])

#### Remove outliers per month using the IQR method

During the data preprocessing step, it is crucial to remove outliers that could negatively impact the accuracy of the model. To do this, the **Interquartile Range (IQR) method** is applied. Outliers are detected and removed if they lie beyond 1.5 times the interquartile range (IQR).

The IQR method operates using the following equation:

$$
IQR = Q_3 - Q_1
$$

Where:
- $Q_1$ is the first quartile (25th percentile).
- $Q_3$ is the third quartile (75th percentile).

Outliers are any data points that fall below the lower bound or above the upper bound:

$$
\text{Lower Bound} = Q_1 - 1.5 \times IQR
$$
$$
\text{Upper Bound} = Q_3 + 1.5 \times IQR
$$

These outliers are removed on a monthly basis to account for seasonal variations in the dataset.

---

In [ ]:
original_count = len(daily_volumes)
trimmed_observed = daily_volumes[daily_volumes['Volume_mcm'] <= maximum_volume]
trimmed_count = len(trimmed_observed)
removed_samples = original_count - trimmed_count
print(f"Number of samples removed: {removed_samples}")
print(f"Valid number of samples: {original_count - removed_samples}")
trimmed_observed

In [ ]:
water_areas_filtered['date'] = pd.to_datetime(water_areas_filtered['date'], errors='coerce')
water_areas_filtered['month'] = water_areas_filtered['date'].dt.month

def calculate_min_surface_area(df, full_volume, minimum_volume, maximum_surface_area):
    volume_percentage_diff = (maximum_volume - minimum_volume) / maximum_volume    
    min_surface_area_estimate =  (maximum_surface_area * (1 - volume_percentage_diff))    
    print(f"Full Supply Volume: {full_volume:.2f} million m³")
    print(f"Minimum Volume: {minimum_volume:.2f} million m³")
    print(f"Full Supply Surface Area: {full_surface_area:.2f} ha")
    print(f"Estimated Minimum Surface Area: {min_surface_area_estimate:.2f} ha")    
    initial_count = len(df)
    df_filtered = df[(df['water_area_ha'] >= min_surface_area_estimate)]
    removed_count = initial_count - len(df_filtered)    
    print(f"Number of values removed based on min and max thresholds: {removed_count}")
    print(f"Remaining samples after threshold filtering: {len(df_filtered)}")
    
    return df_filtered

minimum_volume = trimmed_observed['Volume_mcm'].min()
water_areas_filtered = calculate_min_surface_area(water_areas_filtered, maximum_volume, minimum_volume, maximum_surface_area)

def remove_outliers_per_month(df):
    cleaned_data = df.copy()
    total_initial_samples = len(cleaned_data)
    
    removed_outliers_count = 0
    for month, group in df.groupby('month'):
        Q1 = group['water_area_ha'].quantile(0.25)
        Q3 = group['water_area_ha'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR        
        outliers = group[
            (group['water_area_ha'] < lower_bound) | 
            (group['water_area_ha'] > upper_bound)
        ]
        removed_outliers_count += len(outliers)
        cleaned_data.loc[group.index, 'water_area_ha'] = group[
            (group['water_area_ha'] >= lower_bound) & 
            (group['water_area_ha'] <= upper_bound)
        ]['water_area_ha']
    
    remaining_after_outliers = len(cleaned_data.dropna(subset=['water_area_ha']))    
    print(f"Total initial samples: {total_initial_samples}")
    print(f"Number of samples removed due to outliers: {removed_outliers_count}")
    print(f"Number of samples remaining after outlier removal: {remaining_after_outliers}")
    
    return cleaned_data

water_areas_cleaned = remove_outliers_per_month(water_areas_filtered)
initial_cleaned_count = len(water_areas_cleaned)
water_areas_cleaned.dropna(subset=['water_area_ha'], inplace=True)
final_cleaned_count = len(water_areas_cleaned)

samples_removed_nan = initial_cleaned_count - final_cleaned_count
print(f"Number of samples removed due to NaN: {samples_removed_nan}")
print(f"Number of samples remaining after dropping NaN values: {final_cleaned_count}")
water_areas_cleaned

### Model preparation and Training

#### Calculate Dam Level Based on Surface Area Using Power Coefficient

The **dam level** (or height of water in the dam) can be estimated using a power-law relationship between the surface area and the level. This relationship assumes that as the surface area increases, the dam level increases at a nonlinear rate, which is determined by a power coefficient derived from the reservoir's geometry or rating curve.

The formula for calculating the dam level is:

$$
L = \left( \frac{A}{A_{\text{full}}} \right)^{\frac{1}{n}} \times L_{\text{full}}
$$

Where:
- $L$ is the **calculated dam level** (height of water) in meters.
- $A$ is the current water surface area in hectares.
- $A_{\text{full}}$ is the full surface area of the reservoir in hectares.
- $L_{\text{full}}$ is the dam level at full capacity.
- $n$ is the power coefficient, which is derived from the rating curve and represents how the surface area changes with the dam level.

In this case, the power coefficient $n$ is dynamically calculated from the reservoir's rating curve data, which reflects the relationship between water surface area and dam level. This dynamic approach allows for a more accurate estimation of the reservoir's dam level based on its current surface area.


In [ ]:
def power_law(x, c, n):
    return c * np.power(x, n)

water_level = rating_curve['water_level'].values
volume_mcm = rating_curve['volume_mcm'].values
low_volume_threshold = trimmed_observed['Volume_mcm'].quantile(0.25)
low_volume_mask = (volume_mcm <= low_volume_threshold)
filtered_water_level_low = water_level[low_volume_mask]
filtered_volume_mcm_low = volume_mcm[low_volume_mask]

if len(filtered_water_level_low) >= 2:
    params_low, _ = curve_fit(power_law, filtered_water_level_low, filtered_volume_mcm_low, p0=[1, 1])
    c_fitted_low, n_fitted_low = params_low
    print(f"Low Volume Fit - c: {c_fitted_low}, n: {n_fitted_low}")
else:
    raise ValueError("Not enough data points to fit the low-volume power law.")

mid_high_volume_mask = (volume_mcm > low_volume_threshold)
filtered_water_level_mid_high = water_level[mid_high_volume_mask]
filtered_volume_mcm_mid_high = volume_mcm[mid_high_volume_mask]

if len(filtered_water_level_mid_high) >= 2:
    params_mid_high, _ = curve_fit(power_law, filtered_water_level_mid_high, filtered_volume_mcm_mid_high, p0=[1, 1])
    c_fitted_mid_high, n_fitted_mid_high = params_mid_high
    print(f"Mid-High Volume Fit - c: {c_fitted_mid_high}, n: {n_fitted_mid_high}")
else:
    raise ValueError("Not enough data points to fit the mid-to-high-volume power law.")


#### Use the Power Coefficient for Level Calculation

Once the power coefficient is calculated dynamically from the rating curve, it can be used to estimate the **dam level** based on the current water surface area. This is done using the inverse power law formula:

**Calculate Dam Level from Surface Area**

The dam level can be estimated from the surface area using the following relationship:

$$
L_{\text{calculated}} = \left( \frac{A_{\text{current}}}{A_{\text{full}}} \right)^{\frac{1}{n}} \times L_{\text{full}}
$$

Where:
- $L_{\text{calculated}}$ is the estimated **dam level**.
- $A_{\text{current}}$ is the current water surface area.
- $A_{\text{full}}$ is the full surface area of the dam at maximum capacity.
- $n$ is the power coefficient calculated from the rating curve data.
- $L_{\text{full}}$ is the **full dam level** at maximum capacity.

### Explanation:
- **`water_area_ha`**: The current surface area in hectares.
- **`full_surface_area`**: The surface area of the dam at 100% capacity (maximum water surface area), which is extracted from the dam attributes.
- **`full_dam_level`**: The height of the water body at maximum capacity (when the dam is full).
- **`n_fitted`**: The power coefficient that describes how the surface area changes with dam level. This coefficient is derived from the rating curve using a power-law relationship.

This coefficient is derived from the power-law relationship between the water level at the dam wall and the surface area of the reservoir. It is used to estimate changes in dam level as a function of surface area. By leveraging this coefficient, we can estimate the water level based on the surface area, assuming that the surface area scales predictably with the dam level.

This relationship is particularly useful when direct measurements of water level are unavailable, but surface area data is available (e.g., from satellite imagery).


In [ ]:
# Assert conditions
assert maximum_surface_area > 0, "maximum_surface_area must be greater than 0."
assert maximum_dam_level > 0, "maximum_dam_level must be greater than 0."
assert 'c_fitted_low' in globals() and 'n_fitted_low' in globals(), "Low-volume power-law parameters must be defined."
assert 'c_fitted_mid_high' in globals() and 'n_fitted_mid_high' in globals(), "Mid-to-high-volume power-law parameters must be defined."

# Clean up the water_areas_cleaned DataFrame
if 'date' in water_areas_cleaned.columns:
    water_areas_cleaned = water_areas_cleaned.drop(columns=['date']).copy(deep=True)  # Make a deep copy

if 'date' not in water_areas_cleaned.columns:
    water_areas_cleaned = water_areas_cleaned.reset_index(drop=False).copy(deep=True)  # Make a deep copy after reset_index

if 'date' not in water_areas_cleaned.columns:
    print(f"'date' column not found. Current columns: {water_areas_cleaned.columns}")

# Make a distinct deep copy of the DataFrame to avoid any unwanted views
water_areas_cleaned = water_areas_cleaned.copy(deep=True)

# Function to calculate dam level from surface area
def calculate_level_from_area(area_ha, volume_threshold, c_low, n_low, c_mid_high, n_mid_high):
    if area_ha <= volume_threshold:
        return (area_ha / maximum_surface_area) ** (1 / n_low) * maximum_dam_level
    else:
        return (area_ha / maximum_surface_area) ** (1 / n_mid_high) * maximum_dam_level

# Apply the calculation function to the water_area_ha column
water_areas_cleaned['calculated_level'] = water_areas_cleaned['water_area_ha'].apply(
    lambda area: calculate_level_from_area(area, low_volume_threshold, c_fitted_low, n_fitted_low, c_fitted_mid_high, n_fitted_mid_high)
)

# Make an explicit deep copy of trimmed_observed to avoid any slices
trimmed_observed = trimmed_observed.copy(deep=True)
# Convert 'Date' column in trimmed_observed DataFrame to datetime
trimmed_observed.loc[:, 'Date'] = pd.to_datetime(trimmed_observed['Date'])

# Ensure 'date' is in datetime format in water_areas_cleaned
if 'date' in water_areas_cleaned.columns:
    # Use .loc to modify 'date' safely, ensuring no warning
    water_areas_cleaned.loc[:, 'date'] = pd.to_datetime(water_areas_cleaned['date'])
else:
    raise KeyError("'date' column not found in water_areas_cleaned")

# Merge the two DataFrames on their date columns
merged_data = pd.merge(trimmed_observed, water_areas_cleaned, left_on='Date', right_on='date', how='inner')
merged_data


In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Dam Level
ax.plot(merged_data['Date'], merged_data['Dam_Level'], 
        label='Dam Level', color='blue', linewidth=1)

# Plot Calculated Level
ax.plot(merged_data['Date'], merged_data['calculated_level'], 
        label='Calculated Level', color='red', linewidth=1)

# Set title and labels
ax.set_title('Dam Level vs Calculated Level Over Time')
ax.set_xlabel('Date')
ax.set_ylabel('Level (m)')
plt.xticks(rotation=-45)
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(title='Legend', loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()

# Save the figure as a PNG image
# image_path = "dam_levels_chart_matplotlib.png"
# plt.savefig(image_path, dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


#### Define Features and Target for Model Training

Since we are trying to predict the **dam level** (`Dam_Level`), the target variable needs to reflect the observed dam level.

- **Features**: The input variables used to predict the dam level. In this case, the features include:
  - `calculated_level`: The estimated dam level, calculated using the power-law relationship between volume and dam level.
  - `water_area_ha`: The current water surface area in hectares.

- **Target**: The output variable the model will try to predict, which in this case is the **dam level** (`Dam_Level`).

This is implemented with the following code:

In [ ]:
print("Preparing features and target for model training...")
merged_data['full_volume'] = full_volume
merged_data['full_surface_area'] = full_surface_area
merged_data['full_dam_level'] = full_dam_level
merged_data['maximum_dam_level'] = maximum_dam_level
merged_data['capacity_of_spillway'] = capacity_of_spillway
merged_data['vertical_drop'] = vertical_drop
merged_data['mean_depth'] = mean_depth
merged_data['full_capacity_elevation'] = full_capacity_elevation
merged_data['shoreline_length'] = shoreline_length
merged_data['maximum_volume'] = maximum_volume
merged_data['maximum_surface_area'] = maximum_surface_area


print("saving preprocesed model data...")
merged_data.to_csv("data/preprocess_data.csv")



------------
## Additional information

<b> License </b> The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0).

Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

<b> Contact </b> If you need assistance, please post a question on the [DE Africa Slack channel](https://digitalearthafrica.slack.com/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).

If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

<b> Compatible datacube version </b>

**References:**
- Garcia Andarcia, M., Dickens, C., Silva, P., Matheswaran, K., & Koo, J. (2024). Digital Twin for management of water resources in the Limpopo River Basin: a concept. Colombo, Sri Lanka: International Water Management Institute (IWMI). CGIAR Initiative on Digital Innovation. 4p.
- Gurusinghe, T., Muthuwatta, L., Matheswaran, K., & Dickens, C. (2024). Developing a foundational hydrological model for the Limpopo River Basin using the Soil and Water Assessment Tool Plus (SWAT+). Colombo, Sri Lanka: International Water Management Institute (IWMI). CGIAR Initiative on Digital Innovation. 14p.
- Leitão, P. C., Santos, F., Barreiros, D., Santos, H., Silva, P., Madushanka, T., Matheswaran, K., Mutuwatte, L., Vickneswaran, K., Retief, H., Dickens, C., Garcia Andarcia, M. (2024). Operational SWAT+ Model: Advancing Seasonal Forecasting in the Limpopo River Basin. Colombo, Sri Lanka:  International Water Management Institute (IWMI). CGIAR Initiative on Digital Innovation.
- Mallick, Archita & Ghosh, Surajit & De Sarkar, Kounik & Roy, Sudip. (2024). Reservoir Water Level Forecasting using Deep Learning Technique. Conference: 2024 IEEE India Geoscience and Remote Sensing Symposium

## Project background: 
The CGIAR Digital Innovation Initiative accelerates the transformation towards sustainable and inclusive agrifood systems by generating research-based evidence and innovative digital solutions. It is one of 32 initiatives of CGIAR, a global research partnership for a food-secure future, dedicated to transforming food, land, and water systems in a climate crisis.

### Contributors

**Hugo Retief**  
*Researcher*  
Email: [hugo@award.org.za](mailto:hugo@award.org.za)  

**Surajith Ghosh**  
*Researcher*  
Email: [S.Ghosh@cgiar.org](mailto:S.Ghosh@cgiar.org)  

**Victoria Neema**  
*Earth Observation Scientist*  
Email: [victoria.neema@digitalearthafrica.org](mailto:victoria.neema@digitalearthafrica.org)  

**Kayathri Vigneswaran**  
*Junior Data Scientist*  
Email: [v.kayathri@cgiar.org](mailto:v.kayathri@cgiar.org)  


In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')